In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
random_state = 42 
raw_dataset = pd.read_csv("./original_dataset/processed_data_encoded.csv") #data has X and Y, community 0-9
X = raw_dataset.drop(columns=["BMI", "DR"])
Y = pd.DataFrame(raw_dataset["DR"])
# Slice your data


X_FOR_FOLDS, X_FINAL_TEST, Y_FOR_FOLDS, Y_FINAL_TEST = train_test_split(X, Y, test_size=0.1, random_state=random_state, stratify=Y)
df = pd.concat([X_FOR_FOLDS, Y_FOR_FOLDS], axis=1)
df.reset_index(drop=True, inplace=True)

In [2]:
import torch
import torch.nn as nn
class FeedForwardBlock(nn.Module):
    def __init__(self, in_features, out_features, dropout=None, activation=nn.ReLU6):
        super().__init__()
        layers = [
            nn.Linear(in_features, out_features),
            nn.BatchNorm1d(out_features),
            activation()
        ]
        if dropout and dropout > 0:
            layers.append(nn.Dropout(dropout))
        self.block = nn.Sequential(*layers)
    
    def forward(self, x):
        return self.block(x)
def print_activations(x, name):
    print(f"{name}: min={x.min().item()}, max={x.max().item()}")
    return x
class MyModel(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim, dropout= None, hidden_dim2= None):
        super().__init__()
        
        # A couple of FeedForward blocks
        self.block1 = FeedForwardBlock(input_dim, hidden_dim, dropout= None)
        self.block2 = FeedForwardBlock(hidden_dim, hidden_dim2, dropout = dropout/2)
        self.block3 = FeedForwardBlock(hidden_dim2, output_dim, dropout)

        # Final output layer (could be softmax, sigmoid, or whatever your target is)
        self.output_layer = nn.Linear(output_dim, 1)  # Just in case you're doing regression or binary classification

    def forward(self, x):
        x = self.block1(x)
        # x = print_activations(x, "after block1")
        x = self.block2(x)
        # x = print_activations(x, "after block2")
        x = self.block3(x)
        # x = print_activations(x, "after block3")
        x = self.output_layer(x)  # Final linear layer
        return x

#* Test the model
test_model = MyModel(input_dim = 20,
                     hidden_dim = 64,
                     hidden_dim2 = 16,
                     output_dim = 4,
                     dropout = .2)
print(test_model)


MyModel(
  (block1): FeedForwardBlock(
    (block): Sequential(
      (0): Linear(in_features=20, out_features=64, bias=True)
      (1): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU6()
    )
  )
  (block2): FeedForwardBlock(
    (block): Sequential(
      (0): Linear(in_features=64, out_features=16, bias=True)
      (1): BatchNorm1d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU6()
      (3): Dropout(p=0.1, inplace=False)
    )
  )
  (block3): FeedForwardBlock(
    (block): Sequential(
      (0): Linear(in_features=16, out_features=4, bias=True)
      (1): BatchNorm1d(4, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU6()
      (3): Dropout(p=0.2, inplace=False)
    )
  )
  (output_layer): Linear(in_features=4, out_features=1, bias=True)
)


In [3]:
from sklearn.metrics import f1_score, precision_score, recall_score, accuracy_score, roc_auc_score
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using", device)
def train_and_evaluate(model, criterion, optimiser, scheduler, train_loader, val_loader, epochs=20, patience=5, device=device, threshold = 0.5):
    # if isinstance(model.last_layer(), nn.Sigmoid) and isinstance(criterion, nn.BCEWithLogitsLoss):
    #     raise ValueError("Model output is Sigmoid but criterion is BCEWithLogitsLoss. Please check your model and criterion compatibility.")
    best_val_loss = float('inf')
    best_model_state = None
    wait = 0
    criterion.to(device) #? Move criterion to device
    #* Epoch Training loop for this fold
    for epoch in range(1,epochs+1):
        #* Set model to training mode: essential for dropout and batch norm layers
        model.train()
        running_loss = 0.0 #? loss for this epoch
        #* Mini-batch training loop
        for batch, (inputs, labels) in enumerate(train_loader,start=1):
            optimiser.zero_grad() #? Zero the gradients
            
            
            torch.set_printoptions(threshold=float('inf'))
            
            assert not torch.isnan(inputs).any(), "Input has NaNs"
            assert not torch.isinf(inputs).any(), "Input has Infs"
            outputs = model(inputs) #? Forward pass through the model
            assert not torch.isnan(outputs).any(), "Model output has NaNs"
            assert not torch.isinf(outputs).any(), "Model output has Infs"
            loss = criterion(outputs, labels) #? Calculate loss
            assert not torch.isnan(loss).any(), "Model loss has NaNs"
            loss.backward() #? Backpropagation
            torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
            running_loss += loss.item()
            optimiser.step() #? Update weights
            scheduler.step()
                
        train_loss = running_loss / len(train_loader)
        # print(f"Epoch: {epoch}, training loss: {train_loss:.4f}")
    
        #* Now we evaluate the model on the validation set, to track training vs validation loss
        model.eval() #? Set model to evaluation mode
        with torch.no_grad(): #? No need to track gradients during evaluation
            val_loss = 0.0    
            for batch, (inputs, labels) in enumerate(val_loader,start=1):#! one pass because val_loader batch size is all, if you want to do it in mini-batches, you MUST change the metric calculations to accept mini-batches
                
                outputs = model(inputs)
                # labels = labels.cpu() 
                loss = criterion(outputs, labels)
                val_loss += loss.item() #? Calculate loss
            avg_val_loss = val_loss / len(val_loader)
        loss_ratio = val_loss / train_loss    
        pos_weight = loss_ratio  # or any other function of loss_ratio you choose
    
        # Update criterion with new pos_weight
        criterion = nn.BCEWithLogitsLoss(pos_weight=torch.tensor([pos_weight]).to(device))
        # Early stopping
        if epoch > 30:
            if avg_val_loss < best_val_loss:
                best_val_loss = avg_val_loss
                best_model_state = model.state_dict()
                wait = 0
            elif avg_val_loss*0.95 <= best_val_loss:
                    wait = 0
            else:
                wait += 1
        if wait >= patience:
            print(f"Early stopping triggered at epoch {epoch}, best val loss: {best_val_loss:.4f}")
            break
        print(f"Epoch: {epoch}".ljust(12), f"training loss:{train_loss:.3f}".ljust(16), f"best_val_loss:{best_val_loss:.3f}".ljust(12), f"Val Loss: {avg_val_loss:.3f}", f"Scheduler lr: {scheduler.get_last_lr()}".ljust(50),end="\r")
    #* Use best model to calculate metrics on the validation set
    #! must be outside epoch loop, it comes after the training and cv loop
    model.load_state_dict(best_model_state) #? Load the best model state
    with torch.no_grad():
        for batch, (inputs, labels) in enumerate(val_loader,start=1):#! one pass because val_loader batch size is all, if you want to do it in mini-batches, you MUST change the metric calculations to accept mini-batches
                
                outputs = model(inputs)
                loss = criterion(outputs, labels)
                labels = labels.cpu() 
                # predictions = (torch.sigmoid(outputs) < 0.5).float().cpu().numpy()
                predictions = (torch.sigmoid(outputs) >= threshold).float().cpu().numpy()
                val_loss += loss.item() #? Calculate loss
                
    #! The following should have length equal to fold number           
    accuracy=accuracy_score(labels, predictions) 
    precision=precision_score(labels, predictions, pos_label=1, zero_division=0)
    recall=recall_score(labels, predictions, pos_label=1)
    f1=f1_score(labels, predictions, pos_label=1)
    auc=roc_auc_score(labels, predictions)
    
    return model, accuracy, precision, recall, f1, auc

Using cuda


In [4]:
from Training_Helper_Functions import *
from Preprocessing_Functions import * 
from sklearn.ensemble import IsolationForest
from sklearn.preprocessing import RobustScaler
import numpy as np
import optuna
from torch import optim
def maximise_combined_score(trial):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    print("Using device:", device)
    epochs = 10000
    random_state = 42
    
    # Scaler
    scaler = RobustScaler() # Or StandardScaler() - You can also make this a hyperparameter if you want

    kFolds = FOLDS_GENERATOR(df, n_splits=5, random_state=42,             
                            OD_majority = IsolationForest(contamination=trial.suggest_float("contamination_majority", 0.01, 0.4), random_state=random_state), 
                            OD_minority = IsolationForest(contamination=trial.suggest_float("contamination_minority", 0.01, 0.2), random_state=random_state), 
                            oversampler_first = trial.suggest_categorical("oversampler_first", [False]),
                            synthesizer =  trial.suggest_categorical("synthesizer", ["TVAE", "CTGAN"]),
                            epochs = trial.suggest_int("epochs", 100, 1000, step=100),
                            n_synthetic_data = trial.suggest_int("n_synthetic_data", 1000, 10000, step=1000),
                            scaler=scaler,      
                               ) # Pass outlier models and scaler

    # Model hyperparameters (first-level optimization)
    hidden_dim = trial.suggest_int("hidden_dim", 512, 512, step=32)
    hidden_dim2 = trial.suggest_int("hidden_dim2", 256, 256, step=32)
    output_dim = trial.suggest_int("output_dim", 96, 96, step=2)

    dropout = trial.suggest_float("dropout", 0.0, 0.4)
    threshold = trial.suggest_float("threshold", 0.3, 0.8)
    # dropout = None
    initial_lr = trial.suggest_float("initial_lr", 1e-4, 1e-4, log=True)
    max_lr = trial.suggest_float("max_lr", 1e-4, 1e-4, log=True)
    weight_decay = trial.suggest_float("weight_decay", 1e-6, 1e-3, log=True)


    # Loss function hyperparameters
    criterion_choice = "FocalLoss"
    # Hyperparameter exploration optimization
    if criterion_choice == "BCEWithLogitsLoss":
        pos_weight = trial.suggest_int("pos_weight", 1, 1)
        alpha = None
        gamma = None
    elif criterion_choice == "FocalLoss":
        pos_weight = None
        alpha = trial.suggest_float("alpha", 0.25, 0.75)
        gamma = trial.suggest_float("gamma", 1.0, 5.0)
    else:
        pos_weight = None

    # Initialize lists for metrics across folds
    accuracy_list = []
    precision_list = []
    recall_list = []
    f1_list = []
    auc_list = []

    # Cross-validation loop
    for fold, (train_x, test_x, train_y, test_y) in enumerate(kFolds, start=1):
        print(f"Fold {fold}:")
        # Create DataLoader for current fold
        train_loader, val_loader = fold_to_dataloader_tensor(train_x, test_x, train_y, test_y, batch_size=64,
                                                            device=device)
        # Instantiate and initialize the model
        model = MyModel(input_dim=get_feature_count(train_loader), hidden_dim=hidden_dim, hidden_dim2=hidden_dim2,
                        output_dim=output_dim, dropout=dropout)
        model.to(device)
        model.apply(init_weights)

        # Map the choice to the actual loss function
        criterion = criterion_mapping(criterion_choice, pos_weight, alpha, gamma)
        optimiser = optim.Adam(model.parameters(), lr=initial_lr, weight_decay=weight_decay)
        scheduler = torch.optim.lr_scheduler.CyclicLR(
            optimiser,
            base_lr=1e-5,
            max_lr=max_lr,
            cycle_momentum=False)

        # Train and evaluate the model on the current fold
        model, accuracy, precision, recall, f1, auc = train_and_evaluate(
            model, criterion, optimiser, scheduler, train_loader, val_loader, epochs=epochs, patience=100,
            device=device, threshold=threshold
        )
        print(f"Accuracy: {accuracy:.4f}, precision: {precision:.4f}, recall: {recall:.4f}, f1: {f1:.4f}, auc: {auc:.4f}")
        del model
        del train_loader
        del val_loader

        # Append the metrics from the current fold
        accuracy_list.append(accuracy)
        precision_list.append(precision)
        recall_list.append(recall)
        f1_list.append(f1)
        auc_list.append(auc)
        break

    # Calculate the average metrics across all folds
    avg_accuracy = np.sum(accuracy_list) / len(accuracy_list)
    avg_precision = np.sum(precision_list) / len(precision_list)
    avg_recall = np.sum(recall_list) / len(recall_list)
    avg_f1 = np.sum(f1_list) / len(f1_list)
    avg_auc = np.sum(auc_list) / len(auc_list)

    # Combine metrics into a single "score"
    # combined_score = (avg_f1 + avg_precision + avg_recall + avg_accuracy + avg_auc) / 5
    combined_score = avg_f1

    return combined_score

Using cuda


/opt/conda/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
# import threading
# import optuna
# from optuna_dashboard import run_server

# def start_dashboard():
#     run_server(storage)

# storage = optuna.storages.InMemoryStorage()
# study = optuna.create_study(direction="maximize", storage=storage, study_name="Basic")

# # Start dashboard in a separate thread
# dashboard_thread = threading.Thread(target=start_dashboard, daemon=True)
# dashboard_thread.start()

# # Run optimization
# study.optimize(maximise_combined_score, n_trials=30)

# # After optimization, print results
# print("Best trial:")
# trial = study.best_trial
# print(f"  Combined score: {trial.value}")
# print("  Best hyperparameters:")
# for key, value in trial.params.items():
#     print(f"    {key}: {value}")


In [6]:
import threading
import optuna
from optuna_dashboard import run_server
# !fuser -k 8080/tcp

# Define your persistent storage
storage = "sqlite:///opt6.db"

# Create or load your study
study_name = "optuna6"
try:
    study = optuna.load_study(study_name=study_name, storage=storage)
except KeyError:
    study = optuna.create_study(study_name=study_name, direction="maximize", storage=storage)

# Start Optuna Dashboard in a separate thread
dashboard_thread = threading.Thread(target=lambda: run_server(storage), daemon=True)
dashboard_thread.start()

# Run optimization
# Ensure the 'DR' column exists in the DataFrame
if 'DR' not in df.columns:
    raise KeyError("'DR' column is missing in the DataFrame. Please ensure it is present before running the optimization.")

study.optimize(maximise_combined_score, n_trials=1000)

# Print results
print("Best trial:")
trial = study.best_trial
print(f"  Combined score: {trial.value}")
print("  Best hyperparameters:")
for key, value in trial.params.items():
    print(f"    {key}: {value}")


Using device: cuda
Original class distribution: DR
0.0    4129
1.0     464
Name: count, dtype: int64


Bottle v0.13.2 server starting up (using WSGIRefServer())...
Listening on http://localhost:8080/
Hit Ctrl-C to quit.



After OD, majority: 3508
After OD, minority: 402
Before oversampling & synthetic data: DR 
0.0    3508
1.0     402
Name: count, dtype: int64
No balancing condition applied


/opt/conda/lib/python3.11/site-packages/sdv/single_table/base.py:105: UserWarning: We strongly recommend saving the metadata using 'save_to_json' for replicability in future SDV versions.
  warnings.warn(
Gen. (-3.60) | Discrim. (-0.25): 100%|██████████| 900/900 [12:39<00:00,  1.19it/s]



Applying SMOTENC oversampling...
                Age        Gender     Community          UAlb           Ucr  \
count  20270.000000  20270.000000  20270.000000  20270.000000  20270.000000   
mean      63.727532      0.631722      3.960582     20.331354   3607.456854   
std        6.959672      0.482349      2.807606     39.342070   5239.106526   
min       36.000000      0.000000      0.000000      0.100000      1.000000   
25%       59.000000      0.000000      2.000000      4.184483      4.396464   
50%       64.000000      1.000000      4.000000      9.063473     76.000000   
75%       68.357314      1.000000      6.000000     18.000000   7773.608541   
max       92.000000      1.000000      9.000000    743.200000  21612.000000   

               UACR            TC            TG          TCTG          LDLC  \
count  20270.000000  20270.000000  20270.000000  20270.000000  20270.000000   
mean      23.776713      5.361760      1.432968      5.129870      3.501887   
std       57.0524

...


Applying SMOTENC oversampling...
                Age        Gender     Community          UAlb           Ucr  \
count  17006.000000  17006.000000  17006.000000  17006.000000  17006.000000   
mean      64.088426      0.531695      4.202517     31.226735   3249.790758   
std        6.584280      0.499009      2.947672     50.081098   4791.578947   
min       36.000000      0.000000      0.000000      0.100000      1.000000   
25%       59.541029      0.000000      2.000000      7.851886      5.000000   
50%       64.532528      1.000000      4.000000     15.200000     13.374327   
75%       69.000000      1.000000      7.000000     30.493021   5906.883464   
max       92.000000      1.000000      9.000000    509.800000  21612.000000   

               UACR            TC            TG          TCTG          LDLC  \
count  17006.000000  17006.000000  17006.000000  17006.000000  17006.000000   
mean      36.217419      5.262489      1.519368      4.625401      3.290117   
std       68.4226

[I 2025-04-20 04:18:57,932] Trial 19 finished with value: 0.288135593220339 and parameters: {'contamination_majority': 0.31934634909443355, 'contamination_minority': 0.19847624714754336, 'oversampler_first': False, 'synthesizer': 'TVAE', 'epochs': 800, 'n_synthetic_data': 6000, 'hidden_dim': 512, 'hidden_dim2': 256, 'output_dim': 96, 'dropout': 0.1890473137698285, 'threshold': 0.310353568379071, 'initial_lr': 0.0001, 'max_lr': 0.0001, 'weight_decay': 0.00032951332417461177, 'alpha': 0.7487986308480048, 'gamma': 4.832263121016547}. Best is trial 10 with value: 0.31140350877192985.


Early stopping triggered at epoch 101, best val loss: 0.0299
Accuracy: 0.7076, precision: 0.1910, recall: 0.5862, f1: 0.2881, auc: 0.6537
Using device: cuda
Original class distribution: DR
0.0    4129
1.0     464
Name: count, dtype: int64
After OD, majority: 2519
After OD, minority: 384
Before oversampling & synthetic data: DR 
0.0    2519
1.0     384
Name: count, dtype: int64
No balancing condition applied


/opt/conda/lib/python3.11/site-packages/sdv/single_table/base.py:105: UserWarning: We strongly recommend saving the metadata using 'save_to_json' for replicability in future SDV versions.
  warnings.warn(
Loss: 15.466:  44%|████▎     | 305/700 [00:31<00:40,  9.73it/s]
[W 2025-04-20 04:20:00,646] Trial 20 failed with parameters: {'contamination_majority': 0.38992761803719844, 'contamination_minority': 0.17177103086362977, 'oversampler_first': False, 'synthesizer': 'TVAE', 'epochs': 700, 'n_synthetic_data': 3000} because of the following error: KeyboardInterrupt().
Traceback (most recent call last):
  File "/home/jovyan/.local/lib/python3.11/site-packages/optuna/study/_optimize.py", line 197, in _run_trial
    value_or_values = func(trial)
                      ^^^^^^^^^^^
  File "/tmp/ipykernel_3786/3368046877.py", line 17, in maximise_combined_score
    kFolds = FOLDS_GENERATOR(df, n_splits=5, random_state=42,
  File "/home/jovyan/ADL2/Preprocessing_Functions.py", line 197, in FOLDS_GE

KeyboardInterrupt: 